# 3D LIDAR Object Detection using TAO PointPillars

Transfer learning is the process of transferring learned features from one application to another. It is a commonly used training technique where you use a model trained on one task and re-train to use it on a different task. 

Train Adapt Optimize (TAO) Toolkit  is a simple and easy-to-use Python based AI toolkit for taking purpose-built AI models and customizing them with users' own data.

<img align="center" src="https://developer.nvidia.com/sites/default/files/akamai/embedded-transfer-learning-toolkit-software-stack-1200x670px.png" width="1080"> 

 ## Learning Objectives
In this notebook, you will learn how to leverage the simplicity and convenience of TAO to:

* Train a PointPillars model on the KITTI dataset
* Prune the trained model
* Retrain the pruned model to recover lost accuracy
* Run evaluation & inference on the trained model to verify the accuracy
* Export & deploy the model in TensorRT

 ### Table of Contents

 This notebook shows an example use case of PointPillars using Train Adapt Optimize (TAO) Toolkit.

 1. [Set up env variables and map drives](#head-1)
 2. [Prepare dataset and pretrained model](#head-2)<br>
     2.1 [Download the dataset](#head-2-1)<br>
     2.2 [Verify the downloaded dataset](#head-2-2)<br>
     2.3 [Convert dataset to required format](#head-2-3)<br>
 3. [Provide training specification](#head-3)
 4. [Run TAO training](#head-4)
 5. [Evaluate trained models](#head-5)
 6. [Prune trained models](#head-6)
 7. [Retrain pruned models](#head-7)
 8. [Evaluate retrained model](#head-8)
 9. [Visualize inferences](#head-9)

## Connect to a GPU Runtime

1.   Change Runtime type to GPU by Runtime(Top Left tab)->Change Runtime Type->GPU(Hardware Accerlerator)
2.   Then click on Connect (Top Right)


## Mounting Google drive
Mount your Google drive storage to this Colab instance

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

##Setup Python Environment
Setup the environment necessary to run the TAO Networks by running the bash script

In [ ]:
!sh /content/drive/MyDrive/ColabNotebooks/pytorch/setup_env_customops_networks.sh

 ## 1. Set up env variables <a class="anchor" id="head-1"></a>

In [ ]:
# Setting up env variables for cleaner command line commands.
import os

print("Please replace the variables with your own.")
%env KEY=tlt_encode
%env EXPERIMENT_DIR=/results/pointpillars
# defaulted to volatile colab instance memory
# depending on the available storage left between google drive and colab instance, choose the data_dir path
# %env DATA_DIR=/content/data
%env DATA_DIR=/content/drive/MyDrive/pointpillars_data/
%env SPECS_DIR=/content/drive/MyDrive/ColabNotebooks/pytorch/cv_notebooks/pointpillars/specs

# Showing list of specification files.
!ls -rlt $SPECS_DIR

 ## 2. Prepare dataset and pretrained model <a class="anchor" id="head-2"></a>

 We will be using the KITTI detection dataset for the tutorial. To find more details please visit
 http://www.cvlibs.net/datasets/kitti/eval_object.php?obj_benchmark=2d. Please download the KITTI detection images (http://www.cvlibs.net/download.php?file=data_object_image_2.zip), labels(http://www.cvlibs.net/download.php?file=data_object_label_2.zip), velodyne LIDAR pointcloud(http://www.cvlibs.net/download.php?file=data_object_velodyne.zip) and LIDAR calibration file(http://www.cvlibs.net/download.php?file=data_object_calib.zip) to $DATA_DOWNLOAD_DIR.
 
The entire kitti dataset for lidar is about 40GB. If you don't have unlimited google storage or colab pro, then use the script at ColabNotebooks/pytorch/util/obtain_subset.py on your local desktop machine and upload the subset created onto your drive/colab instance storage

The data will then be extracted to have below structure.

```bash
│── ImageSets
│── training
│   ├──calib & velodyne & label_2 & image_2
│── testing
│   ├──calib & velodyne & image_2
```

The `testing` directory will not be used in this notebook as it has no labels. For the `training` dataset, we will have some script to do data preprocessing and split it into `train` and `val` splits. Finally the directory seen by TAO PointPillars should look like below.

```bash
│── train
│   ├──lidar & label
│── val
│   ├──lidar & label
```

You may use this notebook with your own dataset as well. To use this example with your own dataset, please follow the same directory structure as mentioned below.

### 2.1 Download the dataset <a class="anchor" id="head-2-1"></a>

Once you have gotten the download links in your email, please populate them in place of the `KITTI_IMAGES_DOWNLOAD_URL`,  `KITTI_LABELS_DOWNLOAD_URL`, `KITTI_LIDAR_DOWNLOAD_DIR` and `KITTI_CALIB_DOWNLOAD_DIR`. This next cell, will download the data and place in `$DATA_DIR`

Note that images are only required for KITTI dataset in this notebook, but not required for a general dataset that follows TAO PointPillars standard format. The reason that we need images in KITTI dataset is KITTI dataset does not conform with the standard format and some pre-processing are necessary for it. The preprocessing will read each image's size and retrieve only points that are in field-of-view(FOV) of camera from the original LiDAR files. The retrieved FOV-only points will be saved to new LiDAR file for each of the original LiDAR file. This is necessary as KITTI dataset has only labels in the FOV of camera, but no labels for points outside of camera FOV.

In [ ]:
import os
!mkdir -p $DATA_DIR

os.environ["URL_IMAGES"]=KITTI_IMAGES_DOWNLOAD_URL
!if [ ! -f $DATA_DIR/data_object_image_2.zip ]; then wget $URL_IMAGES -O $DATA_DIR/data_object_image_2.zip; else echo "image archive already downloaded"; fi 

os.environ["URL_LABELS"]=KITTI_LABELS_DOWNLOAD_URL
!if [ ! -f $DATA_DIR/data_object_label_2.zip ]; then wget $URL_LABELS -O $DATA_DIR/data_object_label_2.zip; else echo "label archive already downloaded"; fi

os.environ["URL_LIDAR"]=KITTI_LIDAR_DOWNLOAD_URL
!if [ ! -f $DATA_DIR/data_object_velodyne.zip ]; then wget $URL_LIDAR -O $DATA_DIR/data_object_velodyne.zip; else echo "velodyne archive already downloaded"; fi 

os.environ["URL_CALIB"]=CALIB_DOWNLOAD_URL
!if [ ! -f $DATA_DIR/data_object_calib.zip ]; then wget $URL_CALIB -O $DATA_DIR/data_object_calib.zip; else echo "calib archive already downloaded"; fi 

### 2.2 Verify the downloaded dataset <a class="anchor" id="head-2-2"></a>

In [ ]:
# Check the dataset is present
!mkdir -p $DATA_DIR
!if [ ! -f $DATA_DIR/data_object_image_2.zip ]; then echo 'Image zip file not found, please download.'; else echo 'Found Image zip file.';fi
!if [ ! -f $DATA_DIR/data_object_label_2.zip ]; then echo 'Label zip file not found, please download.'; else echo 'Found Labels zip file.';fi
!if [ ! -f $DATA_DIR/data_object_velodyne.zip ]; then echo 'Velodyne zip file not found, please download.'; else echo 'Found Velodyne zip file.';fi
!if [ ! -f $DATA_DIR/data_object_calib.zip ]; then echo 'Calib zip file not found, please download.'; else echo 'Found Calib zip file.';fi

In [ ]:
# unpack 
!unzip -u $DATA_DIR/data_object_image_2.zip -d $DATA_DIR
!unzip -u $DATA_DIR/data_object_label_2.zip -d $DATA_DIR
!unzip -u $DATA_DIR/data_object_velodyne.zip -d $DATA_DIR
!unzip -u $DATA_DIR/data_object_calib.zip -d $DATA_DIR

### 2.3 Convert dataset to required format<a class="anchor" id="head-2-3"></a>

In [ ]:
# Create output directories
!mkdir -p $DATA_DIR/train/lidar
!mkdir -p $DATA_DIR/train/label
!mkdir -p $DATA_DIR/val/lidar
!mkdir -p $DATA_DIR/val/label

In [ ]:
# Retrieve FOV-only LIDAR points from 360-degree LIDAR points
# Since only FOV data is labelled in KITTI dataset
!mkdir /content/drive/MyDrive/pointpillars_data/train/lidar -p
!python $SPECS_DIR/gen_lidar_points.py -p $DATA_DIR/training/velodyne \
                                                           -c $DATA_DIR/training/calib    \
                                                           -i $DATA_DIR/training/image_2  \
                                                           -o $DATA_DIR/train/lidar

In [ ]:
# Convert labels from Camera coordinate system to LIDAR coordinate system, etc
!mkdir /content/drive/MyDrive/pointpillars_data/train/label/
!python $SPECS_DIR/gen_lidar_labels.py -l $DATA_DIR/training/label_2 \
                                                           -c $DATA_DIR/training/calib \
                                                           -o $DATA_DIR/train/label

In [ ]:
# Drop DontCare class
!python $SPECS_DIR/drop_class.py $DATA_DIR/train/label DontCare

In [ ]:
# train/val split
!python $SPECS_DIR/kitti_split.py $SPECS_DIR/val.txt \
                                                      $DATA_DIR/train/lidar \
                                                      $DATA_DIR/train/label \
                                                      $DATA_DIR/val/lidar \
                                                      $DATA_DIR/val/label

In [ ]:
# Generate dataset statistics for data augmentation
!tao pointpillars dataset_convert -e $SPECS_DIR/pointpillars.yaml

 ## 3. Provide training specification <a class="anchor" id="head-3"></a>

In [ ]:
!cat $SPECS_DIR/pointpillars.yaml

 ## 4. Run TAO training <a class="anchor" id="head-4"></a>
 * Provide the sample spec file for training.

In [ ]:
!tao pointpillars -h

In [ ]:
!tao pointpillars train -e $SPECS_DIR/pointpillars.yaml -r $EXPERIMENT_DIR -k $KEY

 ## 5. Evaluate trained models <a class="anchor" id="head-5"></a>

The evaluation metric in TAO PointPillars is different from that in official metric of KITTI point cloud detection. While KITTI metric considers easy/moderate/hard categories of objects and filters small objects whose sizes are smaller than a threshold, it is only meaningful for KITTI dataset. Instead, TAO PointPillars metric is a general metric that does not classify objects into easy/moderate/hard categories and does not exclude objects in calculation of metric. This makes TAO PointPillars metric a general metric that is applicable to a general dataset. The final result is average precision(AP) and mean average precision(mAP) regardless of its details in computation. Due to this, the TAO PointPillars metric is not comparable with KITTI official metric on KITTI dataset, although they should be roughly the same.

In [ ]:
!tao pointpillars evaluate -e $SPECS_DIR/pointpillars.yaml -r $EXPERIMENT_DIR -k $KEY

 ## 6. Prune trained models <a class="anchor" id="head-6"></a>
 * Specify pre-trained model
 * Threshold for pruning
 * A key to save and load the model
 * Output directory to store the model
 
Usually, you just need to adjust `-pth` (threshold) for accuracy and model size trade off. Higher `pth` gives you smaller model (and thus higher inference speed) but worse accuracy. The threshold to use is depend on the dataset. A `pth` value below is just a start point. If the retrain accuracy is good, you can increase this value to get smaller models. Otherwise, lower this value to get better accuracy.

In [ ]:
!tao pointpillars prune -e $SPECS_DIR/pointpillars.yaml -r $EXPERIMENT_DIR -k $KEY \
                    -m $EXPERIMENT_DIR/ckpt/checkpoint_epoch_80.tlt \
                    -pth 0.1

In [ ]:
!ls -lht $EXPERIMENT_DIR

 ## 7. Retrain pruned models <a class="anchor" id="head-7"></a>
 * Model needs to be re-trained to bring back accuracy after pruning
 * Specify re-training specification

In [ ]:
# Retraining using the pruned model as pretrained weights 
!tao pointpillars train -e $SPECS_DIR/pointpillars_retrain.yaml -r $EXPERIMENT_DIR/retrain -k $KEY

In [ ]:
# Listing the newly retrained model.
!ls -lht $EXPERIMENT_DIR/retrain/ckpt

 ## 8. Evaluate retrained model <a class="anchor" id="head-8"></a>

In [ ]:
!tao pointpillars evaluate -e $SPECS_DIR/pointpillars_retrain.yaml -r $EXPERIMENT_DIR/retrain -k $KEY

 ## 9. Visualize inferences <a class="anchor" id="head-9"></a>
 In this section, we run the inference command on the trained models.

In [ ]:
!tao pointpillars inference -e $SPECS_DIR/pointpillars_retrain.yaml -r $EXPERIMENT_DIR/retrain -k $KEY

The `inference` command will produce visualization of bounding boxes of objects in and rendering of point cloud. This command can be slow due to plots. If you are not going to finish it, you can abort it and check the partial detected results(images) in `$USER_EXPERIMENT_DIR/retrain/detected_boxes`.